### import

In [2]:
require("h2o")
h2o.init(nthreads = 3, max_mem_size = "8g")

Loading required package: h2o

----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


Attaching package: 'h2o'

The following objects are masked from 'package:stats':

    cor, sd, var

The following objects are masked from 'package:base':

    %*%, %in%, &&, ||, apply, as.factor, as.numeric, colnames,
    colnames<-, ifelse, is.character, is.factor, is.numeric, log,
    log10, log1p, log2, round, signif, trunc



 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         46 minutes 51 seconds 
    H2O cluster version:        3.10.5.3 
    H2O cluster version age:    1 year and 16 days !!! 
    H2O cluster name:           H2O_started_from_R_user_uan385 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   7.10 GB 
    H2O cluster total cores:    4 
    H2O cluster allowed cores:  3 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    R Version:                  R version 3.4.3 (2017-11-30) 


Warning message in h2o.clusterInfo():
"
Your H2O cluster version is too old (1 year and 16 days)!
Please download and install the latest version from http://h2o.ai/download/"

### 내장 dataset 사용

In [3]:
data(iris)
iris_h2o <- as.h2o(iris, destination_frame = "iris_h2o")
h2o.ls()

  |======================================================================| 100%


key
glm_model
iris_h2o
iris_h2o_imp
modelmetrics_glm_model@4024457045165199856_on_train_iris_h2o@6088677615796984851
test_iris_h2o
train_iris_h2o


In [4]:
class(iris_h2o)

[1] "H2OFrame"

In [5]:
str(iris_h2o)

Class 'H2OFrame' <environment: 0x000000000c51cf28> 
 - attr(*, "op")= chr "Parse"
 - attr(*, "id")= chr "iris_h2o"
 - attr(*, "eval")= logi FALSE
 - attr(*, "nrow")= int 150
 - attr(*, "ncol")= int 5
 - attr(*, "types")=List of 5
  ..$ : chr "real"
  ..$ : chr "real"
  ..$ : chr "real"
  ..$ : chr "real"
  ..$ : chr "enum"
 - attr(*, "data")='data.frame':	10 obs. of  5 variables:
  ..$ Sepal.Length: num  5.1 4.9 4.7 4.6 5 5.4 4.6 5 4.4 4.9
  ..$ Sepal.Width : num  3.5 3 3.2 3.1 3.6 3.9 3.4 3.4 2.9 3.1
  ..$ Petal.Length: num  1.4 1.4 1.3 1.5 1.4 1.7 1.4 1.5 1.4 1.5
  ..$ Petal.Width : num  0.2 0.2 0.2 0.2 0.2 0.4 0.3 0.2 0.2 0.1
  ..$ Species     : Factor w/ 3 levels "setosa","versicolor",..: 1 1 1 1 1 1 1 1 1 1


### csv로 불러오기

In [6]:
irisPath <- system.file("extdata", "prostate_folder", package = "h2o")
iris_h2o_imp <- h2o.importFile(irisPath, destination_frame = "iris_h2o_imp")
h2o.ls()

  |======================================================================| 100%


key
glm_model
iris_h2o
iris_h2o_imp
modelmetrics_glm_model@4024457045165199856_on_train_iris_h2o@6088677615796984851
test_iris_h2o
train_iris_h2o


### dataset 나누기

In [9]:
set.seed(1234)
train_idx <- sample(1:nrow(iris), size = 0.7 * nrow(iris), replace = FALSE)
train_iris <- iris[train_idx, ]
test_iris <- iris[-train_idx, ]

In [10]:
with(train_iris, prop.table(table(Species)))

Species
    setosa versicolor  virginica 
 0.3714286  0.2761905  0.3523810 

In [11]:
with(test_iris, prop.table(table(Species)))

Species
    setosa versicolor  virginica 
 0.2444444  0.4666667  0.2888889 

In [12]:
train_iris_h2o <- as.h2o(train_iris, "train_iris_h2o")
test_iris_h2o <- as.h2o(test_iris, "test_iris_h2o")
target <- "Species"
features <- names(train_iris)[!names(train_iris) %in% target]

  |======================================================================| 100%
  |======================================================================| 100%


### model

In [13]:
glm_model <- h2o.glm(x = features, y = target, training_frame = train_iris_h2o, 
    model_id = "glm_model", family = "multinomial")

summary(glm_model)

  |======================================================================| 100%
Model Details:

H2OMultinomialModel: glm
Model Key:  glm_model 
GLM Model: summary
       family        link                                regularization
1 multinomial multinomial Elastic Net (alpha = 0.5, lambda = 9.091E-4 )
  number_of_predictors_total number_of_active_predictors number_of_iterations
1                         15                          11                   27
  training_frame
1 train_iris_h2o

H2OMultinomialMetrics: glm
** Reported on training data. **

Training Set Metrics: 

Extract training frame with `h2o.getFrame("train_iris_h2o")`
MSE: (Extract with `h2o.mse`) 0.0136285
RMSE: (Extract with `h2o.rmse`) 0.1167412
Logloss: (Extract with `h2o.logloss`) 0.05376167
Mean Per-Class Error: 0.02050326
Null Deviance: (Extract with `h2o.nulldeviance`) 229.0628
Residual Deviance: (Extract with `h2o.residual_deviance`) 11.28995
R^2: (Extract with `h2o.r2`) 0.9811617
AIC: (Extract with `h2o.aic`

In [14]:
pred_iris_glm <- as.data.frame(h2o.predict(glm_model, newdata = test_iris_h2o))
test_iris$pred_glm <- pred_iris_glm$predict
with(test_iris, table(Species, pred_glm, dnn = c("Real", "Predict")))

  |======================================================================| 100%


            Predict
Real         setosa versicolor virginica
  setosa         11          0         0
  versicolor      0         20         1
  virginica       0          0        13

In [15]:
rf_model <- h2o.randomForest(x = features, y = target, training_frame = train_iris_h2o, 
    model_id = "rf_model", ntrees = 100)

summary(rf_model)

  |======================================================================| 100%
Model Details:

H2OMultinomialModel: drf
Model Key:  rf_model 
Model Summary: 
  number_of_trees number_of_internal_trees model_size_in_bytes min_depth
1             100                      300               47067         1
  max_depth mean_depth min_leaves max_leaves mean_leaves
1         6    3.10000          2         10     4.81000

H2OMultinomialMetrics: drf
** Reported on training data. **
** Metrics reported on Out-Of-Bag training samples **

Training Set Metrics: 

Extract training frame with `h2o.getFrame("train_iris_h2o")`
MSE: (Extract with `h2o.mse`) 0.03387537
RMSE: (Extract with `h2o.rmse`) 0.1840526
Logloss: (Extract with `h2o.logloss`) 0.432699
Mean Per-Class Error: 0.05250078
Confusion Matrix: Extract with `h2o.confusionMatrix(<model>,train = TRUE)`)
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
           setosa versicolor virginica  Error      Rate
setosa    

In [16]:
pred_iris_rf <- as.data.frame(h2o.predict(rf_model, newdata = test_iris_h2o))
test_iris$pred_rf <- pred_iris_rf$predict
with(test_iris, table(Species, pred_rf, dnn = c("Real", "Predict")))

  |======================================================================| 100%


            Predict
Real         setosa versicolor virginica
  setosa         11          0         0
  versicolor      0         20         1
  virginica       0          2        11

In [18]:
gbm_model <- h2o.gbm(x = features, y = target, training_frame = train_iris_h2o, 
    model_id = "gbm_model", ntrees = 100)

summary(gbm_model)

  |======================================================================| 100%
Model Details:

H2OMultinomialModel: gbm
Model Key:  gbm_model 
Model Summary: 
  number_of_trees number_of_internal_trees model_size_in_bytes min_depth
1             100                      300               58374         1
  max_depth mean_depth min_leaves max_leaves mean_leaves
1         5    4.59667          2          9     7.80000

H2OMultinomialMetrics: gbm
** Reported on training data. **

Training Set Metrics: 

Extract training frame with `h2o.getFrame("train_iris_h2o")`
MSE: (Extract with `h2o.mse`) 8.902443e-07
RMSE: (Extract with `h2o.rmse`) 0.0009435276
Logloss: (Extract with `h2o.logloss`) 0.0003444433
Mean Per-Class Error: 0
Confusion Matrix: Extract with `h2o.confusionMatrix(<model>,train = TRUE)`)
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
           setosa versicolor virginica  Error      Rate
setosa         39          0         0 0.0000 =  0 / 39
versico

In [19]:
pred_iris_gbm <- as.data.frame(h2o.predict(gbm_model, newdata = test_iris_h2o))
test_iris$pred_gbm <- pred_iris_gbm$predict
with(test_iris, table(Species, pred_gbm, dnn = c("Real", "Predict")))

  |======================================================================| 100%


            Predict
Real         setosa versicolor virginica
  setosa         11          0         0
  versicolor      0         20         1
  virginica       0          1        12